In [1]:
import pandas as pd
from psifr import fr
import numpy as np

cdcatbeh = pd.read_csv('../../data/cdcatbeh_psifr.csv')

# move item string representation to a separate column
cdcatbeh['item_string'] = cdcatbeh['item']
cdcatbeh['item_string_index'] = fr.pool_index(cdcatbeh['item_string'], pd.unique(cdcatbeh['item_string'])).values.astype(int)

# make sure position column is coded as an integer
cdcatbeh['position'] = cdcatbeh['position'].values.astype(int)

# reindex items based on initual study position
for each in cdcatbeh.groupby(['subject', 'list']):
    cdcatbeh.loc[(cdcatbeh.subject==each[0][0]) &( cdcatbeh.list==each[0][1]), 'item'] = fr.pool_index(
    each[1]['item'], pd.unique(each[1]['item']))

merged = fr.merge_free_recall(cdcatbeh, list_keys=['category', 'list_type', 'item_string', 'item_string_index'])
merged = merged.drop(columns=['prior_list', 'prior_input'])
merged = merged.dropna(axis='index', subset='input')
merged.to_csv('../../data/CDCATBEH.csv', index=False)

# # re index so all trials ascribed to one subject
# subject_index = 1
# trial_index = 0
# for each in cdcatbeh.groupby(['subject', 'list']):
#     trial_index += 1
#     cdcatbeh.loc[(cdcatbeh.subject==each[0][0]) & (cdcatbeh.list==each[0][1]), 'list'] = trial_index
# cdcatbeh['subject'] = subject_index

# agg_merged = fr.merge_free_recall(cdcatbeh, list_keys=['category', 'list_type', 'item_string', 'item_string_index'])
# agg_merged = agg_merged.drop(columns=['prior_list', 'prior_input'])
# agg_merged.to_csv('../../data/AggCDCATBEH.csv', index=False)

merged

c:\ProgramData\Miniconda3\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


,subject,list,item,input,output,study,recall,repeat,intrusion,category,list_type,item_string,item_string_index
0,1,1,0.0,1.0,1.0,True,True,0,False,cel,light,Halle Berry,0
1,1,1,1.0,2.0,3.0,True,True,0,False,cel,light,Simon Cowell,1
2,1,1,2.0,3.0,4.0,True,True,0,False,cel,light,Kristen Stewart,2
3,1,1,3.0,4.0,8.0,True,True,0,False,obj,light,highlighter,3
4,1,1,4.0,5.0,NaN,True,False,0,False,obj,light,acorn,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19994,97,8,22.0,23.0,15.0,True,True,0,False,cel,light,Halle Berry,0
19995,97,8,23.0,24.0,6.0,True,True,0,False,cel,light,Kristen Stewart,2
19996,97,8,24.0,25.0,13.0,True,True,0,False,obj,light,brick,114
19997,97,8,25.0,26.0,12.0,True,True,0,False,obj,light,staple remover,62


In [42]:
cdcatmr_wiki2vec = pd.read_csv('../../data/cdcatmr-wiki2vec.csv')

items = pd.unique(merged['item_string'])
wordpool = np.zeros((len(items)+1, 300))
keys = cdcatmr_wiki2vec['item'].values.tolist()

for item in items:
    i = merged.loc[merged['item_string'] == item, 'item_string_index'].values[0]
    wordpool[i] = cdcatmr_wiki2vec.loc[keys.index(item)].values[2:]
    
np.save('../../data/cdcatmr-wiki2vec.npy', wordpool)

In [43]:
cdcatmr_use = pd.read_csv('../../data/cdcatmr-use.csv')
items = pd.unique(merged['item_string'])
wordpool = np.zeros((len(items)+1, 512))
keys = cdcatmr_use['itemName'].values.tolist()

for item in items:
    i = merged.loc[merged['item_string'] == item, 'item_string_index'].values[0]
    wordpool[i] = cdcatmr_use.loc[keys.index(item)].values[3:]

np.save('../../data/cdcatmr-use.npy', wordpool)

In [59]:
cdcatmr_use = pd.read_csv('../../data/cdcatmr-use.csv')
items = pd.unique(merged['item_string'])
wordpool = np.zeros((len(items)+1, 3))
keys = cdcatmr_use['itemName'].values.tolist()

for item in items:
    i = merged.loc[merged['item_string'] == item, 'item_string_index'].values[0]
    category_index = cdcatmr_use.loc[keys.index(item)].values[2]-1
    wordpool[i, category_index] = 1

np.save('../../data/cdcatmr-cat.npy', wordpool)